In [26]:
import tweepy
import pandas as pd
import datetime as datetime

auth = tweepy.OAuth2BearerHandler("AAAAAAAAAAAAAAAAAAAAAH6nYwEAAAAAK%2BezpbKj7NUWoAMW6SbFOfFruD4%3Dj134xtdV6I1kcGl4a1wnAaa7N2W0hQCA9avu7sDyvEEVWjIwmy")
api = tweepy.API(auth)

In [20]:
#Searching the archive for tweets mentioning the mavs

mavsSearch = api.search_full_archive('dev', 'mavs', maxResults = 100)

In [ ]:
#Viewing the first tweet

import json
status = mavsSearch[0]
json_str = json.dumps(status._json)
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

In [78]:
#Creates the mineTweet function which extracts the information from a query and returns a dictionary with tweet data
#BUG: hashtags label also collects other random data from entities

def mineTweets(query, fromDate=datetime.datetime.now()):
        data = []
        fromDate = datetime.datetime.strptime(str(fromDate), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')

        searchResult = api.search_full_archive('dev', query, fromDate=fromDate, maxResults = 100)
        for item in searchResult:
                mined = {
                        'tweet_id':        item.id,
                        'name':            item.user.name,
                        'screen_name':     item.user.screen_name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.text,
                        'hashtags':        item.entities['hashtags'],
                        'location':        item.place,
                        'is_verified':     item.user.verified,
                        'created_at':      item.created_at
                }
                try:
                        mined['retweet_text'] = item.retweeted_status.full_text
                except:
                        mined['retweet_text'] = 'None'
                try:
                        mined['quote_text'] = item.quoted_status.full_text
                        mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                        mined['quote_text'] = 'None'
                        mined['quote_screen_name'] = 'None'
                
                data.append(mined)
        return pd.DataFrame(data)

In [91]:
mavs_df = mineTweets('mavs', '202202100019')
mavs_df = pd.DataFrame()

time = datetime.datetime.now()
time = datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')
time = int(time)

while time > 202202090019:
    mavs_df = mavs_df.append(mineTweets('mavs', time))
    
    dtTime = datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
    if mavs_df.tweet_id.duplicated().sum() > 0:
        dtTime = dtTime - datetime.timedelta(minutes=1)
    else:
        dtTime = dtTime - datetime.timedelta(minutes=5)
    time = int(dtTime.strftime('%Y%m%d%H%M'))

mavs_df

TooManyRequests: 429 Too Many Requests
Exceeded rate limit